# GeoPython Notebook

Welcome to the GeoPython Workshop!

This workshop aims to present the Python programming as the main toolset of a Geo Data Scientist. The workshop will include exercises and problems where you will analyze and manipulate geo data. This workshop requires prior knowledge of the Python programming language and some knowledge of Geo domain.

This workshop is for users who are familiar with the Python programming language. If you want to have a quick preview on getting started with Python with Geodata please refer to this [link](https://github.com/lkpanganiban/geopython-tutorial)

## Data Acquistion and Data Reading

Geo data can be stored anywhere which ranges from databases, APIs, file based proprietary formats and other internal systems. In this section, we will take a look at some of the libraries in which you can use to extract information from these different sources

### Filebased Geo Data: *SV

Comma delimited, tab delimited and space delimited; these formats or data usually comes from the spreadsheets or other applications which generates logs or sensor data. We can open these types of data using the `csv` library in Python which is part of its standard library.

In [1]:
import csv # let's use an external library to read csv files
with open('data/top10sites.csv', 'r') as csvfile: # this reads the csv file
    reader = csv.DictReader(csvfile) # this converts each row into a python dictionary.
    for row in reader:
        print(row)
        # print(row["Place"]) # you can access data using a certain key

OrderedDict([('id', '1'), ('Max Value', '1327512.36221'), ('Place', 'Katanapan Point-Cagayan Valley'), ('Longitude', '121.156'), ('Latitude', '19.392')])
OrderedDict([('id', '2'), ('Max Value', '1262860.15324'), ('Place', 'Katanapan Point-Cagayan Valley'), ('Longitude', '121.156'), ('Latitude', '19.4')])
OrderedDict([('id', '3'), ('Max Value', '1253266.53637'), ('Place', 'Cabudadan-Cagayan Valley'), ('Longitude', '121.444'), ('Latitude', '19.352')])
OrderedDict([('id', '4'), ('Max Value', '1244616.1615'), ('Place', 'Cape Bojeador-Ilocos'), ('Longitude', '119.996'), ('Latitude', '18.936')])
OrderedDict([('id', '5'), ('Max Value', '1223701.93723'), ('Place', 'Cape Bojeador-Ilocos'), ('Longitude', '119.988'), ('Latitude', '18.936')])
OrderedDict([('id', '6'), ('Max Value', '1197975.16676'), ('Place', 'Katanapan Point-Cagayan Valley'), ('Longitude', '121.148'), ('Latitude', '19.408')])
OrderedDict([('id', '7'), ('Max Value', '1186722.44419'), ('Place', 'Cabudadan-Cagayan Valley'), ('Longit

### Filebased Geo Data: Raster/Image Formats

These types of files are usually the satellite images or interpolated continuous data. You can think of them as an array which represents numerical values of that particular data. You must also take into consideration the orientation of the image and origin of the array, some origin start from the upper left hand corner and some on lower right. We are going to utilize rasterio to read the raster image and manipulate them using numpy

In [2]:
import numpy
import rasterio # rasterio is used to read and write raster images

# Register format drivers from GDAL like tiff images
with rasterio.open('data/bathymetry.tif') as src:
    bathymetry = src.read() # rasterio automatically reads the whole image as an multi-dimensional array
    print(type(bathymetry))
    # print(src.width)
    # print(src.height)
    # print(src.bounds)
    # print(src.crs)

<class 'numpy.ndarray'>


### Filebased Geo Data: Vector Formats

These types of files came from some GIS processing. Similar to Raster formats, they contain some metadata which can aid you to process them properly. We will use fiona to read some shapefiles!

In [3]:
from fiona import collection
from shapely.geometry import Point, shape
with collection("data/top10sites_buffer.shp", "r") as input:
    for point in input:
        print(point) # this displays a geojson file
        # print(shape(point['geometry']))

{'type': 'Feature', 'id': '0', 'properties': OrderedDict([('name', 'Katanapan Point-Cagayan Valley')]), 'geometry': {'type': 'Polygon', 'coordinates': [[(121.656, 19.392), (121.6535923633361, 19.34299142983522), (121.64639264020163, 19.294454838991935), (121.63447016786611, 19.24685766137277), (121.61793976625565, 19.200658283817454), (121.59696063217419, 19.156301631587002), (121.57173480615128, 19.1142148834902), (121.54250522668137, 19.074803357918178), (121.50955339059328, 19.038446609406726), (121.47319664208183, 19.00549477331863), (121.43378511650981, 18.97626519384873), (121.391698368413, 18.95103936782582), (121.34734171618256, 18.930060233744356), (121.30114233862724, 18.913529832133896), (121.25354516100808, 18.901607359798383), (121.2050085701648, 18.8944076366639), (121.156, 18.892), (121.10699142983523, 18.8944076366639), (121.05845483899195, 18.901607359798383), (121.01085766137278, 18.913529832133896), (120.96465828381746, 18.930060233744356), (120.92030163158701, 18.95

### Remote Geo Data: APIs - Non W*S

Most web services have their own set of APIs or application programming interfaces in which you can call. Some require keys to access them. Most Geo APIs may not follow the typical Open Geospatial Consortium (OGC) standards so you need to have extra care when dealing with these remote services. 

In [4]:
import requests

point_endpoint = 'https://api.orbit.phl-microsat.xyz/v1/point/41463/20170815T0100Z'

results = requests.get(point_endpoint).json()
results
# results ['data']
# results ['data']['properties']

{'data': {'type': 'Feature',
  'geometry': {'type': 'Point',
   'coordinates': [121.06525330263776, 14.206951421751079]},
  'properties': {'elevation': 378298.84375, 'timestamp': 1502758800.0}},
 'meta': {'tle': {'catalog_id': 41463,
   'line1': '1 41463U 98067HT  17226.86213157  .00006355  00000-0  74091-4 0  9993',
   'line2': '2 41463  51.6411  89.0410 0001139  76.9154  10.4807 15.63104425 73973',
   'query_time': '2017-08-15T01:00:00Z',
   'epoch': '2017-08-14T20:41:28Z'}}}

### Remote Geo Data: APIs - W*S 

The following library OWSLib accesses the OGC compliant API or webservice.

In [5]:
from owslib.wms import WebMapService
wms = WebMapService('https://www.gebco.net/data_and_products/gebco_web_services/web_map_service/mapserv?',version='1.1.1')
# wms.identification.type
# wms.identification.title
# list(wms.contents)
# wms['GEBCO_Grid'].boundingBox

/mnt/c/Users/lkp/Development/pista_ng_mapa/env/lib/python3.6/site-packages/owslib/map/wms111.py:127: UserWarning: Content metadata for layer "GEBCO_LATEST" already exists. Using child layer
  warnings.warn('Content metadata for layer "%s" already exists. Using child layer' % cm.id)
/mnt/c/Users/lkp/Development/pista_ng_mapa/env/lib/python3.6/site-packages/owslib/map/wms111.py:127: UserWarning: Content metadata for layer "GEBCO_08_GRID" already exists. Using child layer
  warnings.warn('Content metadata for layer "%s" already exists. Using child layer' % cm.id)


## Data Cleanups and Data Processing

There are a lot of geoprocessing workflows that you can use and you have to decide in your organization which works. You should also consider the limitations of your data and the product/workflow.

### Handling Missing Tabular Data

When we are handling tabular data, we often have some missing values. We can remove those data or we can assign a `0` value. Pandas is a good pre-processing and data janitoring tool to handle these types of data.

Pandas has a wide variety of drivers for reading various structured data (csv, spreadsheets, sql results, etc.). In the following section, we are going to read a CSV file.

In [6]:
import pandas as pd

In [7]:
df = pd.read_csv("data/top10sites - missing.csv")

The `head()` function displays the first N rows of the data. For the reverse of head you can use `tail()` to display the last N rows.

In [8]:
df.head(11)

,id,Max Value,Place,Longitude,Latitude
0,1,1.327512e+06,Katanapan Point-Cagayan Valley,121.156,19.392
1,2,1.262860e+06,Katanapan Point-Cagayan Valley,121.156,19.400
2,3,1.253267e+06,Cabudadan-Cagayan Valley,121.444,19.352
3,4,1.244616e+06,Cape Bojeador-Ilocos,119.996,18.936
4,5,1.223702e+06,Cape Bojeador-Ilocos,119.988,18.936
5,6,1.197975e+06,Katanapan Point-Cagayan Valley,121.148,19.408
6,7,1.186722e+06,Cabudadan-Cagayan Valley,121.444,19.344
7,8,1.179810e+06,Katanapan Point-Cagayan Valley,121.148,19.400
8,9,1.148528e+06,Katanapan Point-Cagayan Valley,121.180,19.384
9,10,1.127088e+06,NaN,119.964,18.944


We can remove missing data using `dropna()`. This will remove the entire row. So be careful when removing data since this may change your result in the end.

In [9]:
no_missing_data = df.dropna()
no_missing_data

,id,Max Value,Place,Longitude,Latitude
0,1,1.327512e+06,Katanapan Point-Cagayan Valley,121.156,19.392
1,2,1.262860e+06,Katanapan Point-Cagayan Valley,121.156,19.400
2,3,1.253267e+06,Cabudadan-Cagayan Valley,121.444,19.352
3,4,1.244616e+06,Cape Bojeador-Ilocos,119.996,18.936
4,5,1.223702e+06,Cape Bojeador-Ilocos,119.988,18.936
5,6,1.197975e+06,Katanapan Point-Cagayan Valley,121.148,19.408
6,7,1.186722e+06,Cabudadan-Cagayan Valley,121.444,19.344
7,8,1.179810e+06,Katanapan Point-Cagayan Valley,121.148,19.400
8,9,1.148528e+06,Katanapan Point-Cagayan Valley,121.180,19.384


If you're not comfortable in removing data, you can fill the empty or null values by a certain value. This will add 

In [10]:
filled_missing_data = df.fillna(0)
filled_missing_data

,id,Max Value,Place,Longitude,Latitude
0,1,1.327512e+06,Katanapan Point-Cagayan Valley,121.156,19.392
1,2,1.262860e+06,Katanapan Point-Cagayan Valley,121.156,19.400
2,3,1.253267e+06,Cabudadan-Cagayan Valley,121.444,19.352
3,4,1.244616e+06,Cape Bojeador-Ilocos,119.996,18.936
4,5,1.223702e+06,Cape Bojeador-Ilocos,119.988,18.936
5,6,1.197975e+06,Katanapan Point-Cagayan Valley,121.148,19.408
6,7,1.186722e+06,Cabudadan-Cagayan Valley,121.444,19.344
7,8,1.179810e+06,Katanapan Point-Cagayan Valley,121.148,19.400
8,9,1.148528e+06,Katanapan Point-Cagayan Valley,121.180,19.384
9,10,1.127088e+06,0,119.964,18.944


### Adjusting Raster Images

When we are dealing with raster data especially datasets which came from sensors, sometime you need to apply corrections and adjustments to its values. We will use numpy to apply the corrections

In [18]:
bathymetry.shape

(1, 2002, 1602)

In [13]:
median = numpy.median(bathymetry)
median

-2288.0

In [16]:
bathymetry[0][bathymetry[0] == 32767] = 0
bathymetry

array([[[    0,     0,     0, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        ...,
        [    0,     0,     0, ..., -1666, -1619, -1568],
        [    0,     0,     0, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0]]], dtype=int16)